In [2]:
## Required packages for calculations and plotting
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [3]:
# Thermal expansion, haline contraction and true scale factor
alph = 1.7657*10**-4
bet = 7.5544*10**-4

volnorming = 10**15 #normalising coeffcients
areanorming = 10**12 #normalising coeffcients
ST_scale=bet/alph

# Establish basic constants 
yr2sec = 365.25*24*60*60
Cp=4000
rho=1024
S0=35

dyrs = 9
init_early = 1979
init_late = 2006
Early_period = (np.array([init_early,init_early+dyrs]) - init_early)*12
Late_period = (np.array([init_late,init_late+dyrs]) - init_early)*12
range_yrs = init_late-init_early+1

In [13]:
ACCESS_mapped_masks = xr.open_mfdataset('Masks_processed/Masks_EN4_108*.nc')
fuzz_combined = ACCESS_mapped_masks.fuzz

mask_ACCESS_xr = xr.open_mfdataset('Basin_mask/mask_EN4.nc')
mask_ACCESS = (mask_ACCESS_xr.__xarray_dataarray_variable__.values)

ACCESS_BSP_data = xr.open_mfdataset('BSP_processed/BSP_ACCESS_TS_hist_fixV*.nc')
Basins = ACCESS_BSP_data.Basin.values
SA_early =  ACCESS_BSP_data.S_mean.isel(Time=slice(Early_period[0],Early_period[1])).mean('Time')

In [14]:
if SF:
    Opt_result = xr.open_mfdataset('Optimisation_results/Optimal_result_ACCESS_SF_79_06.nc')
else:
    Opt_result = xr.open_mfdataset('Optimisation_results/Optimal_result_ACCESS_freerun.nc')
dT_mixing = Opt_result.dT_mixing.values.reshape(Basins.size,SA_early.shape[-1])
dS_mixing = Opt_result.dS_mixing.values.reshape(Basins.size,SA_early.shape[-1])
dS_adj = Opt_result.dS_adjustment.values.reshape(Basins.size,SA_early.shape[-1])
dT_adj = Opt_result.dT_adjustment.values.reshape(Basins.size,SA_early.shape[-1])

## Remove NaNs from the visualisation
dT_mixing[np.isnan(dT_mixing)] = 0
dS_mixing[np.isnan(dS_mixing)] = 0
dS_adj[np.isnan(dS_adj)] = 0
dT_adj[np.isnan(dT_adj)] = 0

In [15]:
fuzz_combined.shape

(128, 42, 173, 360)

In [7]:
dT_mix_tot = 0
dS_mix_tot = 0
dT_adj_tot = 0
dS_adj_tot = 0
g_ij_tot = 0

for i in tqdm(range(SA_early.shape[-1])):
    tmp_dTmix = 0
    tmp_dSmix = 0
    tmp_dTadj = 0
    tmp_dSadj = 0
    tmp_gij = 0
    for j in range(Basins.size):
        tmp = dT_mixing[j,i]*mask_ACCESS_xr.__xarray_dataarray_variable__[j,:,:,:]*fuzz_combined[i,:,:,:]
        tmp_dTmix = tmp+tmp_dTmix
        tmp = dS_mixing[j,i]*mask_ACCESS_xr.__xarray_dataarray_variable__[j,:,:,:]*fuzz_combined[i,:,:,:]
        tmp_dSmix = tmp+tmp_dSmix
        tmp = dT_adj[j,i]*mask_ACCESS_xr.__xarray_dataarray_variable__[j,:,:,:]*fuzz_combined[i,:,:,:]
        tmp_dTadj = tmp+tmp_dTadj
        tmp = dS_adj[j,i]*mask_ACCESS_xr.__xarray_dataarray_variable__[j,:,:,:]*fuzz_combined[i,:,:,:]
        tmp_dSadj = tmp+tmp_dSadj
        # tmp = section_trans[j,i]*mask_ACCESS_xr.__xarray_dataarray_variable__[j,:,:,:]*fuzz_combined[i,:,:,:]
        # tmp_gij = tmp+tmp_gij

    dT_mix_tot = tmp_dTmix+dT_mix_tot
    dS_mix_tot = tmp_dSmix+dS_mix_tot
    dT_adj_tot = tmp_dTadj+dT_adj_tot
    dS_adj_tot = tmp_dSadj+dS_adj_tot
    g_ij_tot = tmp_gij+g_ij_tot

  0%|          | 0/128 [00:00<?, ?it/s]

IndexError: Index is not smaller than dimension 1 >= 1

In [42]:
if SF:
    dT_mix_tot.to_netcdf('Remapped_props/dT_mix_Eulerian_ACCESS_SF_79_06.nc')
    dS_mix_tot.to_netcdf('Remapped_props/dS_mix_Eulerian_ACCESS_SF_79_06.nc')
    dT_adj_tot.to_netcdf('Remapped_props/dT_adj_Eulerian_ACCESS_SF_79_06.nc')
    dS_adj_tot.to_netcdf('Remapped_props/dS_adj_Eulerian_ACCESS_SF_79_06.nc')
    # g_ij_tot.to_netcdf('Remapped_props/g_ij_Eulerian_ACCESS_SF_79_06.nc')
else:
    dT_mix_tot.to_netcdf('Remapped_props/dT_mix_Eulerian_ACCESS_freerun.nc')
    dS_mix_tot.to_netcdf('Remapped_props/dS_mix_Eulerian_ACCESS_freerun.nc')
    dT_adj_tot.to_netcdf('Remapped_props/dT_adj_Eulerian_ACCESS_freerun.nc')
    dS_adj_tot.to_netcdf('Remapped_props/dS_adj_Eulerian_ACCESS_freerun.nc')
    # g_ij_tot.to_netcdf('Remapped_props/g_ij_Eulerian_ACCESS_freerun.nc')
